# Data Preparation

### Load outliers

In [1]:
import pandas as pd
import numpy as np

# Manually collected list of outlying countries
outliers_data = pd.read_csv('./data/outliers.csv', dtype={
  'country': 'string',
  'independent': 'object',
  'dependent': 'object',
  'syllabi_count': 'Int64',
  'reasoning': 'string',
  'outlying_start_year': 'Int64',
  'outlying_end_year': 'Int64',
  'supporting_papers': 'string',
  'supporting_syllabi': 'string'
}).dropna().reset_index(drop=True)

# Convert variable names to lists
outliers_data['dependent'] = outliers_data['dependent'].map(lambda val: val.split('; '))
outliers_data['independent'] = outliers_data['independent'].map(lambda val: val.split('; '))
outliers_data['id'] = None

for i, outlier_case in outliers_data.iterrows():
  outliers_data.loc[i, 'id'] = f"{outlier_case['country_code']} {outlier_case['outlying_start_year']} - {outlier_case['outlying_end_year']}"

display(outliers_data)

,country,country_code,independent,dependent,syllabi_count,reasoning,outlying_start_year,outlying_end_year,supporting_papers,supporting_syllabi,id
0,China,CHN,[democracy],[fdi_pc],7,Democratic countries attract more foreign inve...,1992,1995,Choi 2009,"Harvard, Yale (comparative politics reading li...",CHN 1992 - 1995
1,Tunisia,TUN,[is_mena],[democracy],7,MENA countries have typically struggled with m...,2011,2020,Bellin 2013,"Yale (comparative politics reading list), UC S...",TUN 2011 - 2020
2,Costa Rica,CRI,[is_latam],"[log_gdp_pc, fdi_pc]",6,The economies of LATAM countries had been stru...,1949,2015,Sada 2015,"Harvard, Yale (comparative politics reading li...",CRI 1949 - 2015
3,Botswana,BWA,"[cpi, natural_resources]",[gdp_growth],6,The resource curse suggests that developing co...,1990,2006,Iimi 2007,"Yale (comparative politics reading list), UNC ...",BWA 1990 - 2006
4,United States of America,USA,[healthcare_spending_pc],[disability_adjusted_life_years],6,Higher healthcare spending should lead to bett...,2004,2004,"Mor 2022, Karabel and Laurison 2011","Harvard, UC San Diego, UCLA, Ohio State, Upenn...",USA 2004 - 2004
5,United States of America,USA,[log_gdp_pc],"[union_strength, left_representation]",6,Countries with high GDP per capita develop bot...,2004,2011,"Mor 2022, Karabel and Laurison 2011","Harvard, UC San Diego, UCLA, Ohio State, Upenn...",USA 2004 - 2011
6,India,IND,"[female_literacy, is_mena]",[female_workforce_participation],4,Outside of ME/Africa (conservative/patriarchal...,2000,2015,Ghai 2018,"Yale (comparative politics reading list), UNC ...",IND 2000 - 2015
7,Brazil,BRA,[democracy],[military_spending_of_gdp],4,Democracies spend less on military than autocr...,1995,2006,Zaverucha 2009,"Harvard, MIT, UNC Chapel Hill, URochester",BRA 1995 - 2006
8,Mexico,MEX,[democracy],[human_rights],3,Higher democratic development should lead to l...,1948,2008,Brennan et al. 2022,"Harvard, MIT, UCLA",MEX 1948 - 2008
9,Hungary,HUN,[is_eu],[democracy],2,Nearly all EU countries are associated with hi...,2010,2023,"Bogaards 2018, Ágh 2016, Buzogány 2017","Stanford, Harvard",HUN 2010 - 2023


### Create master country dataset
##### Identify necessary variables

In [2]:
temp = outliers_data.explode(column=['independent']).explode(column=['dependent'])
variable_names = pd.concat([temp['independent'], temp['dependent']]).unique().tolist()
variable_names

['democracy',
 'is_mena',
 'is_latam',
 'cpi',
 'natural_resources',
 'healthcare_spending_pc',
 'log_gdp_pc',
 'female_literacy',
 'is_eu',
 'gdp_pc',
 'ethnic_fractionalization',
 'business_ownership_rate',
 'fdi_pc',
 'gdp_growth',
 'disability_adjusted_life_years',
 'union_strength',
 'left_representation',
 'female_workforce_participation',
 'military_spending_of_gdp',
 'human_rights',
 'education',
 'peace',
 'liberal_immigration_policy',
 'unemployment']

#### Import merged dataset

In [3]:
# Delete all previous data transformations and load Master_Data_V2

country_data = pd.read_csv('./data/testing/Master_Data_V6.csv')
country_data['Year'] = country_data['Year'].apply(lambda string: int(string))
country_data = country_data.apply(pd.to_numeric, errors = 'ignore')
display(country_data)

,Country Name,Year,healthcare_spending_of_gdp,healthcare_spending_pc,fdi,natural_resources,gdp,gdp_growth,gdp_pc,gdp_ppp_pc,...,population,union_strength,peace,left_representation,education,log_gdp_pc,fdi_pc,migration_surplus,liberal_immigration_policy,cpi
0,Albania,1970,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2135479.0,-2.114,NaN,-3.8090,NaN,NaN,NaN,11102.0,0.005199,NaN
1,Albania,1971,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2187853.0,-2.114,NaN,NaN,NaN,NaN,NaN,10007.0,0.004574,NaN
2,Albania,1972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2243126.0,-2.114,NaN,NaN,NaN,NaN,NaN,8796.0,0.003921,NaN
3,Albania,1973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2296752.0,-2.114,NaN,NaN,NaN,NaN,NaN,7346.0,0.003198,NaN
4,Albania,1974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2350124.0,-2.114,NaN,-3.8090,NaN,NaN,NaN,5834.0,0.002482,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5780,United States,2018,16.640944,10284.55469,2.150000e+11,0.595679,2.050000e+13,2.945385,62823.30944,62823.30944,...,326838199.0,0.237,4.0,0.5165,NaN,11.048081,657.817846,1200796.0,0.003674,NaN
5781,United States,2019,16.676474,10661.02832,3.160000e+11,0.557098,2.140000e+13,2.294439,65120.39466,65120.39466,...,328329953.0,0.237,NaN,NaN,NaN,11.083993,962.446457,1158444.0,0.003528,NaN
5782,United States,2020,18.815826,11702.40918,1.380000e+11,0.329506,2.110000e+13,-2.767803,63528.63430,63528.63430,...,331501080.0,0.237,NaN,NaN,NaN,11.059246,416.288236,675560.0,0.002038,NaN
5783,United States,2021,NaN,NaN,4.930000e+11,1.279944,2.330000e+13,5.945485,70219.47245,70219.47245,...,331893745.0,0.237,NaN,NaN,NaN,11.159381,1485.415159,561580.0,0.001692,NaN


#### Create a mapping of column names

In [4]:
column_name_mapping = {
  # Currently no columns to map
}

country_data = country_data.rename(columns = column_name_mapping)

#### Internal use: report missing variables

In [5]:
missing = []
for var in variable_names:
  if var not in country_data.columns:
    missing.append(var)

superfluous = []
for var in country_data.columns:
  if var not in variable_names and var not in ['Country Name', 'Year', 'Country Code']:
    superfluous.append(var)

print("Missing variables in the master dataset:")
print(missing)

print("Superfluous variables in the master dataset:")
print(superfluous)

Missing variables in the master dataset:
[]
Superfluous variables in the master dataset:
['healthcare_spending_of_gdp', 'fdi', 'gdp', 'gdp_ppp_pc', 'gini', 'net_migration', 'population', 'migration_surplus']


# Comparison of Outlier Detection Algorithms

### Set up algorithms

In [6]:
# Begin by importing all methods from PyOD (table source: https://pyod.readthedocs.io/en/latest/ 11/5/2023)
methods_table = pd.read_csv('./data/pyod_methods.csv')
methods_table['is_pyod'] = True
methods_table['successful_installation'] = False # Will update once successfully imported
methods_table['constructor'] = None

pyod_blacklist = [
  'MAD', # Only for univariate data
  'LSCP', # Requires to specify parameter "Base unsupervised outlier detectors from PyOD"
  'ABOD', # RuntimeWarning: invalid value encountered in scalar divide ... RuntimeWarning: Degrees of freedom <= 0 for slice
  'FastABOD', # Same error
  'SOS', # RuntimeWarning: overflow encountered in multiply beta[i] = beta[i] * 2.0
  # 'LOCI', # Slow
  # 'KPCA', # Slow
  # 'R-Graph', # Slow
  # 'MO_GAAL', # Slow
  # 'AnoGAN', # Slow
  # 'ALAD', # Slow
  'XGBOD' # Supervised
]

# TODO: Add any other methods we want from lit review, with is_pyod = False,
# to denote that we need custom logic for them in the testing loop (can't call the pyod .fit() API)

# Import all methods programatically
for i, method in methods_table.iterrows():
  if (method['is_pyod'] and method['Abbr'] not in pyod_blacklist):
    method_name = method['Class'].split('.')[-1]
    module_name = method['Class'].replace(f'.{method_name}', '')
    try:
      module = __import__(module_name, globals(), locals(), [method_name])
      method_constructor = vars(module)[method_name]
      print(f'Imported {i} / {len(methods_table)}: {method_constructor}')
      methods_table.loc[i, 'successful_installation'] = True
      methods_table.loc[i, 'constructor'] = method_constructor
    except Exception as e:
      print(f"FAILED {i} / {len(methods_table)}: {method['Class']}. Message: {repr(e)}")

methods_table = methods_table[methods_table['successful_installation'] == True].reset_index(drop = True)
display(methods_table)
print(f"Imported {len(methods_table)} methods")

Imported 0 / 56: <class 'pyod.models.ecod.ECOD'>
Imported 1 / 56: <class 'pyod.models.copod.COPOD'>
Imported 6 / 56: <class 'pyod.models.qmcd.QMCD'>
Imported 7 / 56: <class 'pyod.models.kde.KDE'>
Imported 8 / 56: <class 'pyod.models.sampling.Sampling'>
Imported 9 / 56: <class 'pyod.models.gmm.GMM'>
Imported 10 / 56: <class 'pyod.models.pca.PCA'>
Imported 11 / 56: <class 'pyod.models.kpca.KPCA'>
Imported 12 / 56: <class 'pyod.models.mcd.MCD'>
Imported 13 / 56: <class 'pyod.models.cd.CD'>
Imported 14 / 56: <class 'pyod.models.ocsvm.OCSVM'>
Imported 15 / 56: <class 'pyod.models.lmdd.LMDD'>
Imported 16 / 56: <class 'pyod.models.lof.LOF'>
Imported 17 / 56: <class 'pyod.models.cof.COF'>
Imported 18 / 56: <class 'pyod.models.cof.COF'>
Imported 19 / 56: <class 'pyod.models.cblof.CBLOF'>
Imported 20 / 56: <class 'pyod.models.loci.LOCI'>
Imported 21 / 56: <class 'pyod.models.hbos.HBOS'>
Imported 22 / 56: <class 'pyod.models.knn.KNN'>
Imported 23 / 56: <class 'pyod.models.knn.KNN'>
Imported 24 / 

,Type,Abbr,Algorithm,Year,Class,is_pyod,successful_installation,constructor
0,Probabilistic,ECOD,Unsupervised Outlier Detection Using Empirical...,2022.0,pyod.models.ecod.ECOD,True,True,<class 'pyod.models.ecod.ECOD'>
1,Probabilistic,COPOD,COPOD: Copula-Based Outlier Detection,2020.0,pyod.models.copod.COPOD,True,True,<class 'pyod.models.copod.COPOD'>
2,Probabilistic,QMCD,Quasi-Monte Carlo Discrepancy outlier detection,2001.0,pyod.models.qmcd.QMCD,True,True,<class 'pyod.models.qmcd.QMCD'>
3,Probabilistic,KDE,Outlier Detection with Kernel Density Functions,2007.0,pyod.models.kde.KDE,True,True,<class 'pyod.models.kde.KDE'>
4,Probabilistic,Sampling,Rapid distance-based outlier detection via sam...,2013.0,pyod.models.sampling.Sampling,True,True,<class 'pyod.models.sampling.Sampling'>
5,Probabilistic,GMM,Probabilistic Mixture Modeling for Outlier Ana...,NaN,pyod.models.gmm.GMM,True,True,<class 'pyod.models.gmm.GMM'>
6,Linear Model,PCA,Principal Component Analysis (the sum of weigh...,2003.0,pyod.models.pca.PCA,True,True,<class 'pyod.models.pca.PCA'>
7,Linear Model,KPCA,Kernel Principal Component Analysis,2007.0,pyod.models.kpca.KPCA,True,True,<class 'pyod.models.kpca.KPCA'>
8,Linear Model,MCD,Minimum Covariance Determinant (use the mahala...,1999.0,pyod.models.mcd.MCD,True,True,<class 'pyod.models.mcd.MCD'>
9,Linear Model,CD,Use Cook’s distance for outlier detection,1977.0,pyod.models.cd.CD,True,True,<class 'pyod.models.cd.CD'>


Imported 41 methods


#### Define normalizations

In [7]:
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import robust_scale
from sklearn.preprocessing import scale
from sklearn.preprocessing import normalize

normalizations = {
  'none': lambda x: x,
  'minmax_scale': minmax_scale,
  'robust_scale': robust_scale,
  'scale': scale,
  'normalize': normalize
}

##### Generate variations of OD methods with different normalization methods

In [8]:
methods_table['id'] = None
methods_table['normalization_func'] = None

methods_table_with_normalizations = pd.DataFrame(columns = methods_table.columns)

for i, method in methods_table.iterrows():
  for normalization_name in normalizations:
    method['id'] = f"{method['Abbr']}_{normalization_name}" if normalization_name != 'none' else method['Abbr']
    method['normalization_func'] = normalizations[normalization_name]
    methods_table_with_normalizations.loc[len(methods_table_with_normalizations)] = method

methods_table = methods_table_with_normalizations


### Run tests

For each outlier case in `outliers_data`:

1. Run each algorithm on `country_data`, using only the columns identified in `outliers_data['independent']` versus `outliers_data['dependent']`. Use only the subset of the rows with the correct time range.
2. Record whether the given algorithm correctly identified the case study country as an outlier, and whether any other countries were detected as outliers.
3. Report a matrix, where the rows are algorithms, and the columns are success/failure (or some non-binary measure?) on the given outlier case.

We are reporting results with the levels of observation:

- a) outlier-method (each experiment being one row)
- b) method (columns = case studies tested with the method; average performance of the method on all case studies)
- c) case study (columns = methods ran on the case study; average performance of all methods on case study)

##### Helper functions

In [9]:
from sklearn import metrics
import traceback
from func_timeout import func_timeout, FunctionTimedOut

def print_summary(outlier_i, outlier_case):
  print("\n------\n")
  print(f"RUNNING OUTLIER CASE STUDY {outlier_i + 1}")
  print(f"Identified by: {outlier_case['supporting_papers']}")
  print(f"Outlying country: {outlier_case['country']}, {outlier_case['outlying_start_year']} - {outlier_case['outlying_end_year']}")
  print(f"In terms of: {outlier_case['dependent']}")
  print(f"With respect to: {outlier_case['independent']}")

def df_to_string(df):
  return "; ".join(list(map(lambda x: x.strip(), df.to_string(header = False, index = False, index_names = False).split('\n'))))

# Helper for geographic dummies
def subsetting_variable_present(variables):
  candidates = ['is_eu', 'is_mena', 'is_latam']

  for candidate in candidates:
    if candidate in variables:
      return candidate
    
  return None

def init_model(method):
  model = None
  try:
    model = method['constructor'](verbose = False, random_state = 2023)
  except:
    try:
      model = method['constructor'](verbose = 0, random_state = 2023)
    except:
      try:
        model = method['constructor'](verbose = False)
      except:
        try:
          model = method['constructor'](verbose = 0)
        except:
          model = method['constructor']()
  return model

def correct_nans_and_drop_country_code(X):
  # For each variable: use median of country, if all values empty then use median of dataset

  for country_code in X['Country Code'].unique():
    country_subset_mask = X['Country Code'] == country_code

    for column in X.columns:
      if column != 'Country Code':
        if (len(X[country_subset_mask][column].dropna()) > 0):
          country_subset_column_median = X.loc[country_subset_mask, column].drop(columns = ['Country Code']).median()
          X.loc[country_subset_mask, column] = X.loc[country_subset_mask, column].fillna(country_subset_column_median)
  
  X = X.drop(columns = ['Country Code'])

  return X.fillna(X.median())

##### Main testing loop

In [10]:
from across import across_od

timeout = 120
country_whitelist = [] # Empty = run all countries except those on the blacklist
country_blacklist = ['Vietnam', 'Portugal'] # Missing reliable PISA scores and business ownership rates
print_debug_dfs = True
should_drop_and_subset_on_regional_vars = True
test_ensemble_only = True # Test ACROSS

experiment_level_test_results = pd.DataFrame()
method_result_dictionary = {}
case_study_result_dictionary = {}

# Keep track of methods / outliers that have timed out or error'd on any one test to exclude them in the end
faulty_method_ids = []
faulty_case_study_ids = []

for outlier_i, outlier_case in outliers_data.iterrows():
  outlier_identifier = outlier_case['id']

  if (
    (outlier_case['country'] not in country_whitelist and len(country_whitelist) > 0) or
    (outlier_case['country'] in country_blacklist)
  ):
    faulty_case_study_ids.append(outlier_identifier)
    continue

  print_summary(outlier_i, outlier_case)

  try:
    relevant_variables = ['Country Code', 'Year'] + outlier_case['dependent'] + outlier_case['independent']
    start_year, end_year = outlier_case['outlying_start_year'], outlier_case['outlying_end_year']

    # Verify that all necessarily variables are found in the master dataframe
    missing_vars = []
    all_na_vars = []
    for var_name in relevant_variables:
      target_country_subset = country_data[country_data['Country Code'] == outlier_case['country_code']]
      target_country_subset = target_country_subset.query(f'{start_year} <= Year <= {end_year}')
      
      if var_name not in country_data.columns:
        missing_vars.append(var_name)
      elif len(target_country_subset[var_name].dropna()) == 0:
        all_na_vars.append(var_name)
    if (len(missing_vars) > 0) or (len(all_na_vars) > 0):
      print("SKIPPING due to missing variables")
      print(f"Missing: {missing_vars}")
      print(f"All NaN: {all_na_vars}")
      faulty_case_study_ids.append(outlier_identifier)
      continue

    # Pick the relevant subset of data
    experimental_subset = country_data.loc[:, relevant_variables]
    experimental_subset = experimental_subset.query(f'{start_year} <= Year <= {end_year}')
    experimental_subset['is_empirical_outlier'] = experimental_subset['Country Code'] == outlier_case['country_code']

    if should_drop_and_subset_on_regional_vars:
      # E.g. instead of including LATAM as an OD variable, do 1D OD only on LATAM countries
      subsetting_variable = subsetting_variable_present(relevant_variables)
      if subsetting_variable and outlier_case['country'] != 'India': # For India, MENA is a control, not the main variable
          experimental_subset = experimental_subset[experimental_subset[subsetting_variable] == 1]
          experimental_subset.drop(columns = [subsetting_variable])

    num_empirical_outliers = len(experimental_subset[experimental_subset['is_empirical_outlier']])
    nan_row_count = experimental_subset.shape[0] - experimental_subset.dropna().shape[0]
    print(f"Sample size: {num_empirical_outliers} outliers, {len(experimental_subset)} total data points")
    print(f"{nan_row_count} / {len(experimental_subset)} rows with NaNs ({float(nan_row_count) / len(experimental_subset) * 100.0}%)")
    print()

    if (print_debug_dfs):
      print("Experimental subset:")
      display(experimental_subset)
      print("Target subset:")
      display(experimental_subset[experimental_subset['is_empirical_outlier']])
  except Exception as e:
    faulty_case_study_ids.append(outlier_identifier)
    print("ERROR IN PREPROCESSING")
    print(traceback.format_exc())
    continue

  # Define the relevant numerical variables and resolve NaNs
  test_run = experimental_subset.copy()
  X = test_run.drop(columns = ['Year', 'is_empirical_outlier'])
  X = correct_nans_and_drop_country_code(X)
  X = X.dropna()

  if test_ensemble_only:
    method_name = 'ACROSS'

    # Predict outliers based on X
    predictions, levels = across_od(X).values()
    test_run['is_predicted_outlier'] = predictions
    test_run['outlier_prediction_level'] = levels
  else:
    # Run test algorithms and see if they correctly identify the outlier
    for method_i, method in methods_table.iterrows():
      try:
        method_name = method['id']
        print(f"Test method {method_i + 1} / {len(methods_table)}: {method_name}")
        
        model = init_model(method)

        X = method['normalization_func'](X)

        # Predict outliers based on X
        model = func_timeout(timeout = timeout, func = model.fit, kwargs={ 'X': X })
        test_run['is_predicted_outlier'] = model.labels_
        test_run['outlier_prediction_level'] = model.decision_scores_
      except FunctionTimedOut:
        faulty_method_ids.append(method_name)
        print("TIMED OUT")
      except Exception as e:
        faulty_method_ids.append(method_name)
        print("ERROR")
        print(traceback.format_exc())
  
  try:
    # Check accuracy
    num_correct_classification_of_target_country = len(test_run.query(f"`is_predicted_outlier` == `is_empirical_outlier` and `Country Code` == \"{outlier_case['country_code']}\""))
    num_predicted_outliers = len(test_run.loc[test_run['is_predicted_outlier'] == 1])
    f1 = metrics.f1_score(test_run['is_empirical_outlier'], test_run['is_predicted_outlier'])

    success_on_target_country = float(num_correct_classification_of_target_country) / num_empirical_outliers

    # Report a)
    experiment_level_test_results = pd.concat([experiment_level_test_results, pd.DataFrame({
      'ID': outlier_identifier,
      'Country Code': outlier_case['country_code'],
      'Paper': outlier_case['supporting_papers'],
      'Dependent': '; '.join(outlier_case['dependent']),
      'Independent': '; '.join(outlier_case['independent']),
      'Year Range': f'{start_year} - {end_year}',
      'Algorithm': method_name,
      'Number of empirical outliers': num_empirical_outliers,
      'Target outliers identified': num_correct_classification_of_target_country,
      'Total outliers identified': num_predicted_outliers,
      'Success on target country': success_on_target_country,
      'F1': f1,
      'Identified outliers': df_to_string(test_run.loc[test_run['is_predicted_outlier'] == 1][['Country Code', 'Year']]),
      'Rows with NaNs': nan_row_count,
      'Sample size': len(experimental_subset)
    }, index = [0])], ignore_index = True)

    # Report b)
    if (method_name not in method_result_dictionary):
      method_result_dictionary[method_name] = {}
    method_result_dictionary[method_name][f'{outlier_identifier}: Accuracy on target (Recall)'] = success_on_target_country
    method_result_dictionary[method_name][f'{outlier_identifier}: Additional found'] = num_predicted_outliers
    method_result_dictionary[method_name][f'{outlier_identifier}: F1'] = f1

    # Report c)
    if (outlier_identifier not in case_study_result_dictionary):
      case_study_result_dictionary[outlier_identifier] = {}
    case_study_result_dictionary[outlier_identifier][f'{method_name}: Accuracy on target (Recall)'] = success_on_target_country
    case_study_result_dictionary[outlier_identifier][f'{method_name}: Additional found'] = num_predicted_outliers
    case_study_result_dictionary[outlier_identifier][f'{method_name}: F1'] = f1
  except Exception as e:
    print("ERROR")
    test_run['is_predicted_outlier'].to_csv('./bruh.csv')
    print(traceback.format_exc())
  


print("Done. Faulty methods:")
print(faulty_method_ids)

print("Faulty case studies:")
print(faulty_case_study_ids)


------

RUNNING OUTLIER CASE STUDY 1
Identified by: Choi 2009
Outlying country: China, 1992 - 1995
In terms of: ['fdi_pc']
With respect to: ['democracy']
Sample size: 4 outliers, 461 total data points
109 / 461 rows with NaNs (23.644251626898047%)

Experimental subset:


,Country Code,Year,fdi_pc,democracy,is_empirical_outlier
22,ALB,1992,-6.159458,0.334,False
23,ALB,1993,-17.971752,0.319,False
24,ALB,1994,-16.523587,0.336,False
25,ALB,1995,-21.958828,0.345,False
72,DZA,1992,NaN,0.085,False
...,...,...,...,...,...
5697,ZWE,1995,NaN,0.228,False
5754,USA,1992,118.161192,0.810,False
5755,USA,1993,193.252513,0.811,False
5756,USA,1994,212.597767,0.811,False


Target subset:


,Country Code,Year,fdi_pc,democracy,is_empirical_outlier
1076,CHN,1992,-6.142647,0.057,True
1077,CHN,1993,-19.614915,0.059,True
1078,CHN,1994,-26.670638,0.060,True
1079,CHN,1995,-28.094003,0.061,True


CHECK
461
461

------

RUNNING OUTLIER CASE STUDY 2
Identified by: Bellin 2013
Outlying country: Tunisia, 2011 - 2020
In terms of: ['democracy']
With respect to: ['is_mena']
Sample size: 9 outliers, 81 total data points
0 / 81 rows with NaNs (0.0%)

Experimental subset:


...

CHECK
1965
1965

------

RUNNING OUTLIER CASE STUDY 15
Identified by: Choi 2022, Rogerson 2006
Outlying country: Norway, 1970 - 2013
In terms of: ['liberal_immigration_policy']
With respect to: ['natural_resources']
Sample size: 44 outliers, 5082 total data points
699 / 5082 rows with NaNs (13.754427390791028%)

Experimental subset:


,Country Code,Year,liberal_immigration_policy,natural_resources,is_empirical_outlier
0,ALB,1970,0.005199,NaN,False
1,ALB,1971,0.004574,NaN,False
2,ALB,1972,0.003921,NaN,False
3,ALB,1973,0.003198,NaN,False
4,ALB,1974,0.002482,NaN,False
...,...,...,...,...,...
5771,USA,2009,0.003361,0.743921,False
5772,USA,2010,0.003332,0.968704,False
5773,USA,2011,0.004244,1.235029,False
5774,USA,2012,0.004216,0.776695,False


Target subset:


,Country Code,Year,liberal_immigration_policy,natural_resources,is_empirical_outlier
4018,NOR,1970,0.000000,0.465517,True
4019,NOR,1971,0.000796,0.404786,True
4020,NOR,1972,0.001440,0.387712,True
4021,NOR,1973,0.000857,0.562344,True
4022,NOR,1974,0.001075,0.828774,True
4023,NOR,1975,0.000826,1.621728,True
4024,NOR,1976,0.001186,2.195535,True
4025,NOR,1977,0.001151,2.004234,True
4026,NOR,1978,0.000956,2.656818,True
4027,NOR,1979,0.000692,5.792107,True


CHECK
5082
5082
Done. Faulty methods:
[]
Faulty case studies:
['VNM 1949 - 2015', 'PRT 1984 - 2002']


In [11]:
experiment_level_test_results.to_csv('./output/test_results_by_experiment.csv' if not test_ensemble_only else './output/ensemble_results_by_experiment.csv')

In [12]:
for faulty_method_id in faulty_method_ids:
  method_result_dictionary.pop(faulty_method_id, None)

for method in method_result_dictionary:
  columns = method_result_dictionary[method]
  relevant_accuracy_columns = list(filter(lambda column_name: 'Recall' in column_name, columns))
  relevant_f1_columns = list(filter(lambda column_name: 'F1' in column_name, columns))

  relevant_accuracy_values = list(map(lambda col_name: columns[col_name], relevant_accuracy_columns))
  relevant_f1_values = list(map(lambda col_name: columns[col_name], relevant_f1_columns))

  method_result_dictionary[method][f'Mean Recall'] = np.average(relevant_accuracy_values)
  method_result_dictionary[method][f'Mean F1'] = np.average(relevant_f1_values)

pd.DataFrame.from_dict(method_result_dictionary, orient = 'index').to_csv('./output/test_results_by_algorithm.csv' if not test_ensemble_only else './output/ensemble_results_by_algorithm.csv')

In [13]:
for faulty_case_study_id in faulty_case_study_ids:
  case_study_result_dictionary.pop(faulty_case_study_id, None)

for case_study in case_study_result_dictionary:
  # Drop faulty methods from aggregation
  for column in case_study_result_dictionary[case_study]:
    if column.split(':')[0] in faulty_method_ids:
      case_study_result_dictionary[case_study].pop(column, None)

  columns = case_study_result_dictionary[case_study]

  relevant_accuracy_columns = list(filter(lambda column_name: 'Recall' in column_name, columns))
  relevant_f1_columns = list(filter(lambda column_name: 'F1' in column_name, columns))
  relevant_accuracy_values = list(map(lambda col_name: columns[col_name], relevant_accuracy_columns))
  relevant_f1_values = list(map(lambda col_name: columns[col_name], relevant_f1_columns))

  case_study_result_dictionary[case_study][f'Mean Recall'] = np.average(relevant_accuracy_values)
  case_study_result_dictionary[case_study][f'Median Recall'] = np.median(relevant_accuracy_values)
  case_study_result_dictionary[case_study][f'Best Recall'] = np.max(relevant_accuracy_values)
  case_study_result_dictionary[case_study][f'Mean F1'] = np.average(relevant_f1_values)
  case_study_result_dictionary[case_study][f'Median F1'] = np.median(relevant_f1_values)
  case_study_result_dictionary[case_study][f'Best Recall'] = np.max(relevant_accuracy_values)
  case_study_result_dictionary[case_study][f'Best F1'] = np.max(relevant_f1_values)

pd.DataFrame.from_dict(case_study_result_dictionary, orient = 'index').to_csv('./output/test_results_by_case_study.csv' if not test_ensemble_only else './output/ensemble_results_by_case_study.csv')

In [14]:
successfully_tested_outliers = []
for _, outlier in outliers_data.iterrows():
  if outlier['id'] not in faulty_case_study_ids:
    successfully_tested_outliers.append(outlier)

pd.DataFrame(successfully_tested_outliers).to_csv('./output/successfully_tested_outliers.csv' if not test_ensemble_only else './output/ensemble_successfully_tested_outliers.csv')

In [15]:
failed_method_abbrs = list(map(lambda method_id: method_id.split('_')[0], faulty_method_ids))
successfully_tested_methods = []
temp_method_abbrs = []

for _, method in methods_table.iterrows():
  if (method['Abbr'] not in failed_method_abbrs) and (method['Abbr'] not in temp_method_abbrs):
    successfully_tested_methods.append(method)
    temp_method_abbrs.append(method['Abbr'])

pd.DataFrame(successfully_tested_methods).to_csv('./output/successfully_tested_methods.csv' if not test_ensemble_only else './output/ensemble_successfully_tested_methods.csv')

#### Find extra outliers mentioned by all top 5 models

In [16]:
if not test_ensemble_only:
  results = pd.read_csv('./output/test_results_by_experiment.csv').drop(columns = 'Unnamed: 0')
  top_methods = pd.read_csv('./output/test_results_by_algorithm.csv').rename(columns = {'Unnamed: 0': 'method'}).sort_values(by = 'Mean accuracy', ascending = False)[0:5][['method', 'Mean accuracy']]
  top_method_names = top_methods['method'].to_list()

  # Key = outlier ID, values = experiments ran on it by top 5 methods
  results_of_top_methods = {}
  for _, result in results.iterrows():
    if result['Algorithm'] in top_method_names:
      if result['ID'] not in results_of_top_methods:
        results_of_top_methods[result['ID']] = []
      result['Identified outliers'] = list(map(lambda outlier: outlier.strip(), result['Identified outliers'].split(';')))
      results_of_top_methods[result['ID']].append(result)

  extra_outliers_table = []

  for case in results_of_top_methods:
    experiments = results_of_top_methods[case]
    countries_identified_by_all_algorithms = []
    for outlier in experiments[0]['Identified outliers']:
      if outlier in experiments[1]['Identified outliers'] and outlier in experiments[2]['Identified outliers'] and outlier in experiments[3]['Identified outliers'] and outlier in experiments[4]['Identified outliers']:
        countries_identified_by_all_algorithms.append(outlier)
    extra_outliers_table.append({ 'Case study': case, 'Outliers consistently identified': countries_identified_by_all_algorithms })

  extra_outliers_table = pd.DataFrame(extra_outliers_table)
  extra_outliers_table.to_csv('./output/consistently_identified_extra_outliers.csv')